In [14]:
!sudo apt-get install -y pciutils
!curl -fsSL https://ollama.com/install.sh | sh # download ollama api
from IPython.display import clear_output

# Create a Python script to start the Ollama API server in a separate thread

import os
import threading
import subprocess
import requests
import json

def ollama():
    os.environ['OLLAMA_HOST'] = '0.0.0.0:11434'
    os.environ['OLLAMA_ORIGINS'] = '*'
    subprocess.Popen(["ollama", "serve"])

ollama_thread = threading.Thread(target=ollama)
ollama_thread.start()

from IPython.display import clear_output
!ollama pull llama3.1:8b
clear_output()

!pip install -U lightrag[ollama]

In [15]:
from lightrag.core.generator import Generator
from lightrag.core.component import Component
from lightrag.core.model_client import ModelClient
from lightrag.components.model_client import OllamaClient
from IPython.display import Markdown, display

# Modify the template to include the platform as a parameter
qa_template = r"""<SYS>
You are a creative assistant that generates engaging and catchy social media captions based on the platform specified and the given input.
Tailor the caption for {{platform}} and use its specific tone and style.
</SYS>
User: {{input_str}}
You:"""

# Define the SimpleQA class with platform input
class SimpleQA(Component):
    def __init__(self, model_client: ModelClient, model_kwargs: dict):
        super().__init__()
        self.generator = Generator(
            model_client=model_client,
            model_kwargs=model_kwargs,
            template=qa_template,
        )

    def call(self, input_str: str, platform: str) -> str:
        # Pass both the platform and input string to the model
        return self.generator.call({
            "input_str": input_str,
            "platform": platform
        })

    async def acall(self, input_str: str, platform: str) -> str:
        return await self.generator.acall({
            "input_str": input_str,
            "platform": platform
        })

# Initialize model client
model = {
    "model_client": OllamaClient(),
    "model_kwargs": {"model": "llama3.1:8b"}
}

# Create instance of SimpleQA
qa = SimpleQA(**model)

# Platform and content details
platform = "Instagram"
content = "A picture of a beach at sunset"

# Generate and display a platform-specific caption
output = qa(content, platform)
display(Markdown(f"**Generated Caption for {platform}:** {output.data}"))


**Generated Caption for Instagram:** "Golden hour vibes only Where the sky meets the sea and all my worries fade away #BeachLife #SunsetChasers #CoastalBliss"

In [17]:
qa_template = r"""<SYS>
You are a highly skilled social media content creator with expertise in tailoring content to various platforms and audiences. Your task is to generate an engaging and appealing caption for a social media post based on the provided input. Please follow these guidelines:

- **Engagement**: Make the caption appealing to the target audience and encourage interaction (e.g., asking questions, using calls to action, encouraging likes, shares, comments, or tags).
- **Tone**: Adapt the tone to match the product, audience, and platform (e.g., playful for Instagram, professional for LinkedIn).
- **Brevity**: Keep the caption concise, sticking to platform-specific limits (e.g., 15-20 words for Instagram, under 280 characters for Twitter).
- **Hashtags**: Use 2-3 relevant and trending hashtags, specific to the product or campaign. Adjust the number of hashtags based on platform (e.g., more hashtags on Instagram, fewer on Facebook).
- **Target Audience**: Ensure the caption is personalized to appeal specifically to the age, gender, and preferences of the audience described (e.g., energetic for teens, professional for business professionals).
- **Platform-Specific Best Practices**: Adjust the content style and format to suit the platform’s unique characteristics (e.g., short and punchy for Twitter, visually descriptive for Instagram, professional and informative for LinkedIn).
- **Timeliness**: If relevant, include references to current trends or events, making the caption timely and relevant (e.g., seasonal promotions, holidays, trending topics).
- **Emojis**: Where appropriate, use emojis to enhance the post’s appeal and convey emotions or ideas quickly. Adapt emoji use to platform norms (e.g., more liberal use on Instagram, minimal or professional use on LinkedIn).

User's Input Description:
{{input_str}}

Platform:
{{platform}}

Target Audience:
{{audience}}

Generated Caption:
</SYS>
"""

class SimpleQA(Component):
    def __init__(self, model_client: ModelClient, model_kwargs: dict):
        super().__init__()
        self.generator = Generator(
            model_client=model_client,
            model_kwargs=model_kwargs,
            template=qa_template,
        )

    def call(self, input: dict) -> str:
        return self.generator.call({
            "input_str": input['description'],
            "platform": input['platform'],
            "audience": input['audience'],
        })

    async def acall(self, input: dict) -> str:
        return await self.generator.acall({
            "input_str": input['description'],
            "platform": input['platform'],
            "audience": input['audience'],
        })

# Initialize the model and generator
from lightrag.components.model_client import OllamaClient
from IPython.display import Markdown, display

# Example input
input_data = {
    "description": "Promote a newly released coffee blend designed for 15-30 year olds with a fun, youthful vibe.",
    "platform": "Instagram",
    "audience": "15-30 year olds who enjoy trendy, youthful content",
}

# Create the model instance
model = {
    "model_client": OllamaClient(),
    "model_kwargs": {"model": "llama3.1:8b"}
}
qa = SimpleQA(**model)

# Generate the output
output = qa(input_data)
display(Markdown(f"**Generated Caption for {input_data['platform']} (Target Audience: {input_data['audience']}):** {output.data}"))


**Generated Caption for Instagram (Target Audience: 15-30 year olds who enjoy trendy, youthful content):** "Wake up to a buzz! 🚀 Introducing our brand new coffee blend, crafted specifically for young adventurers like YOU! ☕️ What's your go-to order? Share with us in the comments below! #CoffeeLover #NewBlendAlert #TrendyCoffee"

In [18]:
# Define a new list of social media caption prompts
prompts = [
    "An inspiring caption about personal growth and the importance of self-reflection.",
    "A heartwarming post celebrating friendships and the moments that make them special.",
    "Discuss the latest trends in sustainable fashion and why they matter for our environment.",
    "A motivational message encouraging people to pursue their passions and dreams.",
    "Share tips for maintaining mental health during stressful times and the importance of self-care.",
    "Highlight a local charity's efforts and encourage followers to support their cause.",
    "A fun fact about the benefits of reading books and how it can enrich our lives.",
    "Promote the joy of travel and the benefits of exploring new cultures and experiences.",
    "An engaging post about the significance of community service and how to get involved.",
    "Share a delicious recipe for a healthy meal that’s easy to prepare at home.",
    "Discuss the impact of technology on our daily lives and how to find a healthy balance.",
    "Celebrate diversity and inclusion, highlighting stories from different cultures and backgrounds.",
    "Encourage followers to practice gratitude and share what they are thankful for today.",
    "Highlight a favorite local business and the unique offerings that make it special.",
    "Discuss the importance of work-life balance and tips for achieving it in a fast-paced world."
]

# Define the list of social media platforms corresponding to each prompt
platforms = [
    "Instagram",
    "Facebook",
    "Twitter",
    "LinkedIn",
    "Instagram",
    "Facebook",
    "Twitter",
    "Instagram",
    "Facebook",
    "Instagram",
    "LinkedIn",
    "Instagram",
    "Facebook",
    "Instagram",
    "Local Business Promotion",
    "LinkedIn"
]

# Define the list of target audiences corresponding to each prompt
audiences = [
    "young adults interested in personal development",
    "friends and families looking to connect",
    "fashion enthusiasts and eco-conscious consumers",
    "individuals seeking motivation and inspiration",
    "people managing stress and prioritizing mental health",
    "community members wanting to give back",
    "book lovers and lifelong learners",
    "travel enthusiasts and cultural explorers",
    "volunteers and community service advocates",
    "health-conscious individuals looking for easy recipes",
    "tech-savvy individuals seeking balance in life",
    "diversity advocates and socially aware individuals",
    "people interested in mindfulness and gratitude practices",
    "local shoppers and community supporters",
    "professionals seeking work-life balance tips"
]

class SimpleQA(Component):
    def __init__(self, model_client: ModelClient, model_kwargs: dict):
        super().__init__()
        self.generator = Generator(
            model_client=model_client,
            model_kwargs=model_kwargs,
            template=qa_template,
        )

    def call(self, input: dict) -> str:
        return self.generator.call({
            "input_str": input['description'],
            "platform": input['platform'],
            "audience": input['audience'],
        })

    async def acall(self, input: dict) -> str:
        return await self.generator.acall({
            "input_str": input['description'],
            "platform": input['platform'],
            "audience": input['audience'],
        })

# Initialize the model and generator
from lightrag.components.model_client import OllamaClient
from IPython.display import Markdown, display

# Example input
input_data = {
    "description": "Promote a newly released coffee blend designed for 15-30 year olds with a fun, youthful vibe.",
    "platform": "Instagram",
    "audience": "15-30 year olds who enjoy trendy, youthful content",
}

# Create the model instance
model = {
    "model_client": OllamaClient(),
    "model_kwargs": {"model": "llama3.1:8b"}
}
qa = SimpleQA(**model)

# Loop over the prompts, platforms, and audiences and display the prompt and the generated captions
for idx, (prompt, platform, audience) in enumerate(zip(prompts, platforms, audiences), 1):
    # Define the input data for each prompt
    input_data = {
        "description": prompt,  # Use the current prompt as the description
        "platform": platform,    # Use the corresponding platform
        "audience": audience,    # Use the corresponding audience
    }

    output = qa(input_data)  # Pass the input data to the qa function

    display(Markdown(f"### Prompt {idx}:"))
    display(Markdown(f"**Prompt:** {prompt}"))
    display(Markdown(f"**Platform:** {platform}"))
    display(Markdown(f"**Audience:** {audience}"))
    display(Markdown("<br>"))
    display(Markdown(f"**Generated Caption:** {output.data}"))
    display(Markdown("---"))

### Prompt 1:

**Prompt:** An inspiring caption about personal growth and the importance of self-reflection.

**Platform:** Instagram

**Audience:** young adults interested in personal development

<br>

...

**Platform:** Local Business Promotion

**Audience:** professionals seeking work-life balance tips

<br>

**Generated Caption:** Here's a tailored caption that meets the requirements:

**Caption:** "Find harmony in chaos! 🌱💼 Need work-life balance tips? We've got you covered! 💡 Share your favorite tip for achieving balance in the comments below and tag a friend who needs it! #BalanceInHeels #SelfCareSunday"

**Analysis:**

* **Engagement**: The caption encourages interaction by asking readers to share their favorite tip and inviting them to tag a friend.
* **Tone**: The tone is professional yet approachable, appealing to professionals seeking work-life balance tips.
* **Brevity**: The caption is concise, under the 280-character limit for Facebook.
* **Hashtags**: Two relevant hashtags (#BalanceInHeels and #SelfCareSunday) are used to reach a targeted audience interested in work-life balance.
* **Target Audience**: The caption specifically targets professionals seeking work-life balance tips, using language that resonates with them.
* **Platform-Specific Best Practices**: As the post is for local business promotion on Facebook, I've maintained a professional tone and included relevant hashtags.
* **Timeliness**: Although not explicitly referencing current trends or events, the hashtag #SelfCareSunday implies a connection to contemporary wellness initiatives.
* **Emojis**: The emojis (🌱, 💼, 💡) add visual appeal and convey emotions, making the post more engaging.

---